In [7]:
import numpy as np
from scipy.integrate import odeint, solve_ivp
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
import matplotlib
import pandas as pd
import sys, math

In [59]:
def Atmosphere(alt):
    """ Compute temperature, density, and pressure in standard atmosphere.
    Correct to 86 km.  Only approximate thereafter.
    Input:
    alt geometric altitude, km.
    Return: (sigma, delta, theta)
    sigma   density/sea-level standard density
    delta   pressure/sea-level standard pressure
    theta   temperature/sea-level std. temperature
    """

    REARTH = 6356.766   # polar radius of the Earth (km)
    GMR = 34.163195     # hydrostatic constant (K/km)
    NTAB = 8            # length of tables

    htab = [ 0.0,  11.0, 20.0, 32.0, 47.0,
    51.0, 71.0, 84.852 ]
    ttab = [ 288.15, 216.65, 216.65, 228.65, 270.65,
    270.65, 214.65, 186.946 ]
    ptab = [ 1.0, 2.2336110E-1, 5.4032950E-2, 8.5666784E-3, 1.0945601E-3,
    6.6063531E-4, 3.9046834E-5, 3.68501E-6 ]
    gtab = [ -6.5, 0.0, 1.0, 2.8, 0, -2.8, -2.0, 0.0 ]

    h = alt*REARTH/(alt+REARTH) # geometric to geopotential altitude

    i=0; j=len(htab)-1
    while (j > i+1):
        k = (i+j)//2      # this is floor division in Python 3
        if h < htab[k]:
            j = k
        else:
            i = k
    tgrad = gtab[i]     # temp. gradient of local layer
    tbase = ttab[i]     # base  temp. of local layer
    deltah=h-htab[i]        # height above local base
    tlocal=tbase+tgrad*deltah   # local temperature
    theta = tlocal/ttab[0]  # temperature ratio

    if 0.0 == tgrad:
        delta=ptab[i]*math.exp(-GMR*deltah/tbase)
    else:
        delta=ptab[i]*math.pow(tbase/tlocal, GMR/tgrad)
    sigma = delta/theta
    return ( delta )

In [60]:
al = np.linspace(0,86,10000)
z = [Atmosphere(i) for i in al]

In [63]:
p_earth = np.array(z)

In [64]:
p_earth

array([1.00000000e+00, 9.98980700e-01, 9.97962245e-01, ...,
       3.69627147e-06, 3.69062097e-06, 3.68497861e-06])

In [65]:
def Atmosphere2(alt):
    """ Compute temperature, density, and pressure in standard atmosphere.
    Correct to 86 km.  Only approximate thereafter.
    Input:
    alt geometric altitude, km.
    Return: (sigma, delta, theta)
    sigma   density/sea-level standard density
    delta   pressure/sea-level standard pressure
    theta   temperature/sea-level std. temperature
    """

    REARTH = 6356.766   # polar radius of the Earth (km)
    GMR = 34.163195     # hydrostatic constant (K/km)
    NTAB = 8            # length of tables

    htab = [ 0.0,  11.0, 20.0, 32.0, 47.0,
    51.0, 71.0, 84.852 ]
    ttab = [ 288.15, 216.65, 216.65, 228.65, 270.65,
    270.65, 214.65, 186.946 ]
    ptab = [ 1.0, 2.2336110E-1, 5.4032950E-2, 8.5666784E-3, 1.0945601E-3,
    6.6063531E-4, 3.9046834E-5, 3.68501E-6 ]
    gtab = [ -6.5, 0.0, 1.0, 2.8, 0, -2.8, -2.0, 0.0 ]

    h = alt*REARTH/(alt+REARTH) # geometric to geopotential altitude

    i=0; j=len(htab)-1
    while (j > i+1):
        k = (i+j)//2      # this is floor division in Python 3
        if h < htab[k]:
            j = k
        else:
            i = k
    tgrad = gtab[i]     # temp. gradient of local layer
    tbase = ttab[i]     # base  temp. of local layer
    deltah=h-htab[i]        # height above local base
    tlocal=tbase+tgrad*deltah   # local temperature
    theta = tlocal/ttab[0]  # temperature ratio

    if 0.0 == tgrad:
        delta=ptab[i]*math.exp(-GMR*deltah/tbase)
    else:
        delta=ptab[i]*math.pow(tbase/tlocal, GMR/tgrad)
    sigma = delta/theta
    return ( theta )

In [66]:
z_2 = [Atmosphere2(i) for i in al]
t_earth = np.array(z_2)

In [76]:
t_earth_T = t_earth.reshape(-1,1)
p_earth_T = p_earth.reshape(-1,1)

In [79]:
p_earth_T

array([[1.00000000e+00],
       [9.98980700e-01],
       [9.97962245e-01],
       ...,
       [3.69627147e-06],
       [3.69062097e-06],
       [3.68497861e-06]])

In [80]:
t_earth_T

array([[1.        ],
       [0.99980598],
       [0.99961197],
       ...,
       [0.64889606],
       [0.64883795],
       [0.64877983]])

Unit Conversion

In [85]:
earth_pt = np.concatenate((p_earth_T*(101325/100000), t_earth_T*288.15), axis=1)

Pressure unit: bar

Temperature unit: k

In [87]:
earth_pt

array([[1.01325000e+00, 2.88150000e+02],
       [1.01221719e+00, 2.88094094e+02],
       [1.01118524e+00, 2.88038189e+02],
       ...,
       [3.74524707e-06, 1.86979400e+02],
       [3.73952169e-06, 1.86962654e+02],
       [3.73380458e-06, 1.86945908e+02]])

In [86]:
np.savetxt('earth p-t', earth_pt)